In [36]:

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI

import anthropic
import gradio as gr

In [37]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [38]:
openai = OpenAI()
claude = anthropic.Anthropic()

In [39]:
system_message = "You are a helpful assistant and my name is Adam"

In [40]:
import pyttsx3

def speak_message(message):
    try:
        engine = pyttsx3.init()
        engine.setProperty('rate', 150)
        engine.setProperty('volume', 1.0)
        engine.say(message)
        engine.runAndWait()
    except Exception as e:
        engine.endLoop()
    finally:
        engine.stop()
        




In [41]:
def stream_gpt4omini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [42]:
def stream_gpt4o(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [43]:
def stream_claude3_5Sonnet(prompt):
    result = claude.messages.stream(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [44]:
def stream_claude3_5Haiku(prompt):
    result = claude.messages.stream(
        model="claude-3-5-haiku-20241022",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [45]:
def stream_model(prompt, model):
    if model=="GPT-4o":
        result = stream_gpt4o(prompt)
    elif model=="GPT-4o-mini":
        result = stream_gpt4omini(prompt)
    elif model=="Claude 3.5 Sonnet":
        result = stream_claude3_5Sonnet(prompt)
    elif model=="Claude 3.5 Haiku":
        result = stream_claude3_5Haiku(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [46]:
theme = gr.themes.Soft()


view = gr.Interface(
    fn=stream_model,
    theme=theme,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT-4o","GPT-4o-mini","Claude 3.5 Sonnet", "Claude 3.5 Haiku"], value="DEFAULT" ,label="Select model")],
    outputs=[gr.Markdown(label="Response:")],
    allow_flagging="never"
)
view.launch()

/Users/adamlindberg/Documents/VSCode/llm_engineering/env/lib/python3.12/site-packages/gradio/components/dropdown.py:201: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: DEFAULT or set allow_custom_value=True.
  warnings.warn(
/Users/adamlindberg/Documents/VSCode/llm_engineering/env/lib/python3.12/site-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [47]:
gr.themes.builder()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
